# CTFishPy Ingest Tutorial

This notebook will walk you through how to ingest a multi-fish tiff scan and seperate them into separate dicom files. Make sure to also sort out your metadata.

Please make sure you have ctfishpy installed. If you have to restart or try again you will have to restart the jupyter notebook - this is due to a fault with napari since it is still in alpha.

Before we start let's import ctfishpy and required libraries then initialise the ctreader and lumpfish:


In [2]:
#Import ctfishpy and initialise ctreader
from pathlib2 import Path
import pandas as pd
import numpy as np
import ctfishpy
import napari
import cv2
# from scipy.ndimage import zoom

dataset_path = '/home/ak18001/Data/HDD/uCT/'
ctreader = ctfishpy.CTreader(data_path=dataset_path)
lump = ctfishpy.Lumpfish()

## Step 1 - define paths, names and read scan

Define the path to your tif folder

Change the detection scale to 40 to make the interface quicker

Set the slice range to read first for instance (900,1100) will read 200 slices in the middle of the scan, 
When you are ready to read the whole scan just set slice_range to `None`

Number your fish by editing `fish_nums`

This step will take the longest (10-15 mins) but you only have to do it once

In [3]:
# tif_path = Path("path/to/005-009_tifs") # Make sure this is the tif file and not the projections
tif_path = Path("/home/ak18001/Data/HDD/uCT/MISC/Ingest/dirty/QT_56_60/tifs/") # Make sure this is the tif file and not the projections

# output_path = Path("path/to/output/")
output_path = Path("/home/ak18001/Data/HDD/uCT/MISC/Ingest/clean")

original_scale = 100 # full scale of scan
detection_scale = 20 # detection scale
slice_range = None # slice range, can be a tuple eg (500,600) or None to read the whole scan

dataset_name = 'QT' # eg EK, AK, QT
fish_nums = [56,57,58,59,60] # numbers of fish in old dataset
new_dataset = 'ak'
new_nums = [461,462,463,464,465] # numbers of fish in clean dataset
voxel_size_x = 0.0202360326938826
voxel_size_y = 0.0202360326938826
voxel_size_z = 0.0202360326938826

ct = lump.read_tiff(tif_path, r=slice_range, scale=original_scale)

[CTFishPy] Reading uCT scan: /home/ak18001/Data/HDD/uCT/MISC/Ingest/dirty/QT_56_60/tifs


100%|██████████| 1955/1955 [07:19<00:00,  4.45it/s]


(1955, 1609, 1374)


## Step 2

Use lumpfish to detect tubes at a 40% scale to make the interface faster

This will start a window with all the detection parameters for you to tune

In [4]:
scale_40 = lump.rescale(ct, detection_scale)
# detect tubes
viewer = napari.Viewer(show=False)
circle_dict = lump.detectTubes(viewer, scale_40)

## Step 3

Label the order of the fish - this is usually marked by a sticker or a cap on the tube of the first fish, then go clockwise. If there is a fish in the middle this is the last one.

In [5]:
# label order
viewer = napari.Viewer(show=False)
ordered = lump.labelOrder(viewer, circle_dict)
ordered

[[225, 68, 57], [104, 27, 62], [39, 148, 53], [67, 275, 60], [198, 289, 61]]

## Step 4

choose the output folder to save your scans as .dicoms, this will label them according to `fish_nums` which you set earlier.

Finally crop the fish according to the detection and the order, 
then align each fish so that the dorsal fin is pointing upwards.

This will also create a temporary metadata file for you to fill out.

In [6]:
cropped_cts = lump.crop(ct, ordered, scale=[detection_scale,original_scale])

df = pd.DataFrame(columns = ['n', 'Dataset', 'old_n', 'age', 'age(old)', 'genotype', 'strain',
       'name', 'shape', 'size', 'VoxelSizeX', 'VoxelSizeY', 'VoxelSizeZ',
       're-uCT scan', 'Comments', 'Phantom', 'Scaling Value', 'Arb Value',
       'angle', 'center'], 
                   index = fish_nums)

for i,cropped in enumerate(cropped_cts):
    num = fish_nums[i]
    new_num = new_nums[i]

    spin_viewer = napari.Viewer(show=False)
    angle, center = lump.spin(spin_viewer, cropped)

    final_ct = ctreader.rotate_array(cropped, angle, is_label=False, center=center)
    ctreader.write_dicom(f"{output_path}/DICOMS/{new_dataset}_{new_num}.dcm", final_ct)

    projections = ctreader.make_max_projections(final_ct)
    z,y,x = projections
    cv2.imwrite(f'{output_path}/PROJECTIONS/z_{new_num}.png', z)
    cv2.imwrite(f'{output_path}/PROJECTIONS/y_{new_num}.png', y)
    cv2.imwrite(f'{output_path}/PROJECTIONS/x_{new_num}.png', x)

    print('num and shape', num, cropped.shape)
    print('angle and center', angle, center)

    df.loc[num]['ak_n']         = new_num
    df.loc[num]['Dataset']      = dataset_name
    df.loc[num]['shape']        = final_ct.shape
    df.loc[num]['size']         = final_ct.size
    df.loc[num]['VoxelSizeX']   = voxel_size_x
    df.loc[num]['VoxelSizeY']   = voxel_size_y
    df.loc[num]['VoxelSizeZ']   = voxel_size_z
    df.loc[num]['angle']        = angle
    df.loc[num]['center']       = center

df.to_csv(f"{output_path}/METADATA/{dataset_name}_{fish_nums[0]}-{fish_nums[-1]}_temp_metadata.csv")

num and shape 56 (1955, 570, 570)
angle and center 343 None
num and shape 57 (1955, 620, 620)
angle and center 76 None
num and shape 58 (1955, 530, 530)
angle and center 202 None
num and shape 59 (1955, 600, 600)
angle and center 144 None
num and shape 60 (1955, 610, 610)
angle and center 240 (371.7361343337783, 359.96981432115194)
